<a href="https://colab.research.google.com/github/staghado/3D-ResNeXt/blob/main/Visualization_of_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !wget https://huggingface.co/datasets/chrisxx/laion2b-en-10K-subset/resolve/main/laion2b-en-1K-experts-large.parquet

In [11]:
! mv kaggle.json /root/.kaggle/ && kaggle datasets download -d romainbeaumont/laion400m

100% 45.5G/45.5G [10:18<00:00, 117MB/s]
100% 45.5G/45.5G [10:18<00:00, 78.9MB/s]


In [ ]:
! unzip laion400m.zip

Archive:  laion400m.zip
  inflating: part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet  
  inflating: part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet  

In [ ]:
# alternatively you may want to look at results from paddleocr alone:
#!wget https://huggingface.co/datasets/chrisxx/laion2b-en-10K-subset/resolve/main/laion2b-en-10K-paddleocr.parquet

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import pandas as pd
import ast
import requests
from io import BytesIO
import re
import cv2
import urllib.request
import numpy as np
from PIL import Image as PILImage

# parquet_fname = 'laion2b-en-1K-experts-large.parquet'
parquet_name = 'part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet'
df = pd.read_parquet(parquet_fname, engine='pyarrow')

In [ ]:
df

,SAMPLE_ID,URL,TEXT,HEIGHT,WIDTH,LICENSE,NSFW,similarity,OCR_BBOXES,OCR_EXPERTS,OCR_TEXT
0,2.641080e+12,https://cdn.shopify.com/s/files/1/0017/3621/25...,"Blue Beach Umbrellas, Point Of Rocks, Crescent...",231.0,300.0,?,UNLIKELY,0.395562,[],[],[]
1,1.069682e+12,http://cdn.bmwblog.com/wp-content/uploads/2016...,BMW-M2-M-Performance-Dekor-Long-Beach-Blue-05,120.0,120.0,?,UNLIKELY,0.324732,[],[],[]
2,7.480780e+11,http://ecx.images-amazon.com/images/I/51phsP4a...,Becoming More Than a Good Bible Study Girl: Li...,125.0,125.0,?,UNLIKELY,0.425868,"[[[46.0, 339.0], [470.0, 339.0], [470.0, 394.0...","['trocr-large-stage1', 'trocr-large-stage1', '...","['BECOMING MORE', 'THAN A GOOD BIBLE STUDY CIR..."
3,3.203431e+12,https://www.deltekproducts.com/_resources/imag...,"""Dynabrade 52632 4-1/2"""" Dia. Right Angle Depr...",150.0,100.0,?,UNLIKELY,0.310137,[],[],[]
4,1.517065e+12,http://www.gamersnine.com/file_g9/2014/05/MANE...,MANETTE XBOX ONE,650.0,431.0,?,NSFW,0.317398,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
996,3.253112e+12,https://img-aws.ehowcdn.com/137x77/photos.dema...,How to Open a Hairdressing Salon,137.0,77.0,?,UNLIKELY,0.301545,[],[],[]
997,4.119222e+12,https://static.wixstatic.com/media/1c009e_89d1...,5Pc Set Florian Dining Table w/ 4 Chairs,498.0,331.0,?,UNLIKELY,0.339066,[],[],[]
998,1.525458e+12,http://cache.boston.com/bonzai-fba/Globe_Photo...,mounted police,508.0,334.0,?,UNLIKELY,0.311321,[],[],[]
999,1.831123e+12,https://id-live-03.slatic.net/p/4/image-878256...,3 in 1 - Diaper Bag - Travel Bassinet - Change...,340.0,340.0,?,UNLIKELY,0.317100,[],[],[]


In [ ]:
def parse_list_string(list_string):
    return list_string.strip('[]').replace("'", "").split(',')

def parse_bounding_boxes(input_string):
    input_string = input_string.replace(' ', '')
    pattern = r'\[\[([\d.,-]+)\],\s?\[([\d.,-]+)\],\s?\[([\d.,-]+)\],\s?\[([\d.,-]+)\]\]'
    matches = re.findall(pattern, input_string)
    bounding_boxes = []

    for match in matches:
        box_coords = [list(map(float, coord.split(','))) for coord in match]
        bounding_boxes.append(box_coords)

    return bounding_boxes


def add_bounding_boxes(image_data, bounding_boxes):
    h = image_data.shape[0]
    w = image_data.shape[1]

    if h == w:
        yscale = h/512
        xscale = w/512
        xoffset = 0
        yoffset = 0
    if h > w:
        wscaled = (512/h) * w
        yscale = h/512
        xscale = w/wscaled
        xoffset = -(512 - wscaled)/2
        yoffset = 0
    else:
        hscaled = (512/w)*h
        yscale = h/hscaled
        xscale = w/512
        xoffset = 0
        yoffset = -(512 - hscaled)/2


    # Iterate through the bounding boxes and draw them on the image
    for box in bounding_boxes:
        scaled_box = []
        for point in box:
            x = (point[0]+xoffset)*yscale
            y = (point[1]+yoffset)*yscale
            scaled_box += [[x, y]]
        box = np.array([tuple(map(int, point)) for point in scaled_box], dtype=np.int32)  # Convert coordinates to int and create a NumPy array
        cv2.polylines(image_data, [box], True, (0, 255, 0), 2)  # Green color, thickness 2

    # Convert the image back to PIL format
    image_pil = PILImage.fromarray(cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB))
    return image_pil

def read_image_from_url(url):
    with urllib.request.urlopen(url) as response:
        image_data = response.read()
    image_array = np.asarray(bytearray(image_data), dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    return image

### select a few examples that have been labelled from certain expert

In [ ]:
exp_col = df['OCR_EXPERTS']
handwritten_idcs = []
for idx, exps in enumerate(exp_col):
    #if 'paddleocr' in exps:
    if f'trocr-large-handwritten' in exps: # other options: paddleocr, trocr-large-stage1
        handwritten_idcs += [idx]

In [ ]:
def on_click(idx):
    with output:
        clear_output(wait=True)
        #img = Image(url=df.iloc[idx]['URL'])
        img = read_image_from_url(df.iloc[idx]['URL'])
        img = add_bounding_boxes(img, parse_bounding_boxes(df.iloc[idx]['OCR_BBOXES']))
        display(img)
        ocr_texts = parse_list_string(df.iloc[idx]['OCR_TEXT'])
        ocr_experts = parse_list_string(df.iloc[idx]['OCR_EXPERTS'])
        for ocr_text, ocr_experts in zip(ocr_texts, ocr_experts):
            print("{0: <40} {1: <40}".format(ocr_text, ocr_experts))

def url_to_memory_view(url):
    response = requests.get(url)
    img_data = BytesIO(response.content)
    return memoryview(img_data.getvalue())

def on_thumbnail_click(idx, button):
    on_click(idx)

In [ ]:
buttons = []
for i, idx in enumerate(handwritten_idcs):
    button = widgets.Button(description='', icon='check', layout=widgets.Layout(width='100px', height='100px'))
    button.style.button_color = 'transparent'
    button.on_click(lambda btn, idx=idx: on_thumbnail_click(idx, btn))
    buttons.append(button)

thumbnails = [widgets.HTML(f'<img src="{df.iloc[idx]["URL"]}" width="100" height="100"/>') for idx in handwritten_idcs]

thumbnails_grid = widgets.GridBox([widgets.VBox([thumbnail, button]) for thumbnail, button in zip(thumbnails, buttons)], layout=widgets.Layout(grid_template_columns="repeat(auto-fill, minmax(100px, 1fr))", grid_gap="10px"))

output = widgets.Output()

main_layout = widgets.VBox([thumbnails_grid, output])
display(main_layout)